In [1]:
%load_ext autoreload
%autoreload 2

Note: I used this notebook to get acclimated with the ABIDES exchange agent and order books and validate the matching engine with empirical data

In [3]:
import os
import sys

(parent_folder_path, current_dir) = os.path.split(os.path.abspath(''))
sys.path.append(parent_folder_path)
sys.path.append(os.path.join(parent_folder_path, 'simulator'))
sys.path.append(os.path.join(parent_folder_path, 'equities/data_processing'))

# os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

import numpy as np
import pandas as pd
import random
from typing import List, Tuple
from copy import deepcopy
from tqdm import tqdm
from glob import glob
from decimal import Decimal

from simulator.core import Message
from simulator.markets.order_book import OrderBook
from simulator.markets.orders import LimitOrder, Side, MarketOrder
from equities.data_processing import itch_preproc
from equities.data_processing import itch_encoding

In [4]:
SYMBOL = "X"
TIME = 0


class FakeExchangeAgent:
    def __init__(self):
        self.messages = []
        self.current_time = TIME
        self.mkt_open = TIME
        self.book_logging = None
        self.stream_history = 10

    def reset(self):
        self.messages = []

    def send_message(self, recipient_id: int, message: Message, _: int = 0):
        self.messages.append((recipient_id, message))

    def logEvent(self, *args, **kwargs):
        pass


def setup_book_with_orders(
    bids: List[Tuple[int, List[int]]] = [], asks: List[Tuple[int, List[int]]] = []
) -> Tuple[OrderBook, FakeExchangeAgent, List[LimitOrder]]:
    agent = FakeExchangeAgent()
    book = OrderBook(agent, SYMBOL)
    orders = []

    for price, quantities in bids:
        for quantity in quantities:
            order = LimitOrder(1, TIME, SYMBOL, quantity, Side.BID, price)
            book.handle_limit_order(order)
            orders.append(order)

    for price, quantities in asks:
        for quantity in quantities:
            order = LimitOrder(1, TIME, SYMBOL, quantity, Side.ASK, price)
            book.handle_limit_order(order)
            orders.append(order)

    agent.reset()

    return book, agent, orders

### Exploring book methods

In [49]:
book, agent, orders = setup_book_with_orders(
    bids=[
        (100, [40, 10]),
        (200, [10, 30, 20, 10]),
    ],
    asks=[
        (300, [10, 50, 20]),
        (400, [40, 10]),
        (500, [20]),
    ],
)

In [15]:
vars(book)

{'owner': <__main__.FakeExchangeAgent at 0x7f3e3c94bb60>,
 'symbol': 'X',
 'bids': [<simulator.markets.price_level.PriceLevel at 0x7f3e3c92eae0>,
 'asks': [<simulator.markets.price_level.PriceLevel at 0x7f3e3c92de50>,
 'last_trade': None,
 'book_log2': [],
 'quotes_seen': set(),
 'history': [{'time': 0,
   'type': 'LIMIT',
   'order_id': 0,
   'agent_id': 1,
   'side': 'BID',
   'quantity': 40,
   'price': 100},
  {'time': 0,
   'type': 'LIMIT',
   'order_id': 1,
   'agent_id': 1,
   'side': 'BID',
   'quantity': 10,
   'price': 100},
  {'time': 0,
   'type': 'LIMIT',
   'order_id': 2,
   'agent_id': 1,
   'side': 'BID',
   'quantity': 10,
   'price': 200},
  {'time': 0,
   'type': 'LIMIT',
   'order_id': 3,
   'agent_id': 1,
   'side': 'BID',
   'quantity': 30,
   'price': 200},
  {'time': 0,
   'type': 'LIMIT',
   'order_id': 4,
   'agent_id': 1,
   'side': 'BID',
   'quantity': 20,
   'price': 200},
  {'time': 0,
   'type': 'LIMIT',
   'order_id': 5,
   'agent_id': 1,
   'side': 'BI

In [16]:
vars(agent)

{'messages': [],
 'current_time': 0,
 'mkt_open': 0,
 'book_logging': None,
 'stream_history': 10}

In [20]:
# ordered by end of bid side book to end of ask side book
orders

[(Agent 1 @ 1970-01-01 00:00:00) : BID 40 X @ $1.00,
 (Agent 1 @ 1970-01-01 00:00:00) : BID 10 X @ $1.00,
 (Agent 1 @ 1970-01-01 00:00:00) : BID 10 X @ $2.00,
 (Agent 1 @ 1970-01-01 00:00:00) : BID 30 X @ $2.00,
 (Agent 1 @ 1970-01-01 00:00:00) : BID 20 X @ $2.00,
 (Agent 1 @ 1970-01-01 00:00:00) : BID 10 X @ $2.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 10 X @ $3.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 50 X @ $3.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 20 X @ $3.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 40 X @ $4.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 10 X @ $4.00,
 (Agent 1 @ 1970-01-01 00:00:00) : ASK 20 X @ $5.00]

In [50]:
print(orders[1])
print(book.get_l3_bid_data())

# Cancel bid order, mid book
book.cancel_order(orders[1])
print(book.get_l3_bid_data())

agent.messages

(Agent 1 @ 1970-01-01 00:00:00) : BID 10 X @ $1.00
[(200, [10, 30, 20, 10]), (100, [40, 10])]
[(200, [10, 30, 20, 10]), (100, [40])]


[(1,
  OrderCancelledMsg(message_id=39, order=(Agent 1 @ 1970-01-01 00:00:00) : BID 10 X @ $1.00))]

In [53]:
print("L1 bid data:", book.get_l1_bid_data()) # best bid tuple (price, volume)
print("L1 ask data:", book.get_l1_ask_data()) # best ask tuple (price, volume)
print("L2 bid data:", book.get_l2_bid_data()) # entire bid side
print("L2 ask data:", book.get_l2_ask_data()) # entire ask side
print("L3 bid data:", book.get_l3_bid_data()) # full order view bid side
print("L3 ask data:", book.get_l3_ask_data()) # full order view ask side

print("transacted volume", book.get_transacted_volume()) # tuple (bid, ask)
print("order book imbalance", book.get_imbalance()) # tuple (volume imbalance value, side that has more volume)

L1 bid data: (200, 70)
L1 ask data: (300, 80)
L2 bid data: [(200, 70), (100, 40)]
L2 ask data: [(300, 80), (400, 50), (500, 20)]
L3 bid data: [(200, [10, 30, 20, 10]), (100, [40])]
L3 ask data: [(300, [10, 50, 20]), (400, [40, 10]), (500, [20])]
transacted volume (0, 0)
order book imbalance (0.2666666666666667, <Side.ASK: 'ASK'>)


In [266]:
# init new book under nasdaq agent
agent = FakeExchangeAgent()
book = OrderBook(agent, SYMBOL)
assert book.bids == book.asks == [] # empty book

# insert bid order
bid_order = LimitOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=10,
    side=Side.BID,
    limit_price=100,
)
book.handle_limit_order(bid_order)

# insert another bid order with id specified
order2 = LimitOrder(
    order_id=69,
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=20,
    side=Side.BID,
    limit_price=100,
)
book.handle_limit_order(order2)

# insert ask order
ask_order = LimitOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=20,
    side=Side.ASK,
    limit_price=110,
)
book.handle_limit_order(ask_order)

print("L3 bid data:", book.get_l3_bid_data())
print("L3 ask data:", book.get_l3_ask_data())

print("bid_order.order_id:", agent.messages[0][1].order.order_id)
print("order2.order_id:", agent.messages[1][1].order.order_id)
print("ask_order.order_id:", agent.messages[2][1].order.order_id)

L3 bid data: [(100, [10, 20])]
L3 ask data: [(110, [20])]
bid_order.order_id: 6
order2.order_id: 69
ask_order.order_id: 7


In [267]:
# send buy order that partially consumes one order
market_order = MarketOrder(
    agent_id=2,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=10,
    side=Side.BID,
)
book.handle_market_order(market_order)

print("L3 bid data:", book.get_l3_bid_data())
print("L3 ask data:", book.get_l3_ask_data())
print("transacted volume", book.get_transacted_volume()) # tuple (buy, sell)

L3 bid data: [(100, [10, 20])]
L3 ask data: [(110, [10])]
transacted volume (10, 0)


In [268]:
# Modify order quantity down from 10 to 30
modified_order = deepcopy(bid_order)
modified_order.quantity = 30

book.modify_order(bid_order, modified_order)
print("L3 bid data:", book.get_l3_bid_data()) # order priority is changed
print("L3 ask data:", book.get_l3_ask_data())

L3 bid data: [(100, [20, 30])]
L3 ask data: [(110, [10])]


In [269]:
# place price-to-comply order and then cancel it
order = LimitOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=10,
    side=Side.BID,
    is_price_to_comply=True,
    limit_price=100,
)
book.handle_limit_order(order)
print("(pre-cancel) L3 bid data:", book.get_l3_bid_data())
print("(pre-cancel) L3 ask data:", book.get_l3_ask_data())

book.cancel_order(order)
print("(post-cancel) L3 bid data:", book.get_l3_bid_data())
print("(post-cancel) L3 ask data:", book.get_l3_ask_data())

(pre-cancel) L3 bid data: [(101, []), (100, [20, 30, 10])]
(pre-cancel) L3 ask data: [(110, [10])]
(post-cancel) L3 bid data: [(100, [20, 30])]
(post-cancel) L3 ask data: [(110, [10])]


In [270]:
# place price-to-comply order and then fill it
order = LimitOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=10,
    side=Side.ASK,
    is_price_to_comply=True,
    limit_price=110,
)
book.handle_limit_order(order)
print("L3 bid data:", book.get_l3_bid_data())
print("L3 ask data:", book.get_l3_ask_data())

market_order = MarketOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=10,
    side=Side.BID,
)
book.handle_market_order(market_order)
print("L3 bid data:", book.get_l3_bid_data())
print("L3 ask data:", book.get_l3_ask_data())

L3 bid data: [(100, [20, 30])]
L3 ask data: [(109, []), (110, [10, 10])]
L3 bid data: [(100, [20, 30])]
L3 ask data: [(110, [10])]


In [271]:
# confirm that the order was executed as intended
agent.messages[-1][1].order.fill_price
print("The order was executed at the inside-ask price:", agent.messages[-1][1].order.fill_price)

The order was executed at the inside-ask price: 109


In [272]:
# Replace 10 @ $110 with 50 @ $111, ask side of book
new_ask_order = LimitOrder(
    agent_id=1,
    time_placed=TIME,
    symbol=SYMBOL,
    quantity=50,
    side=Side.ASK,
    limit_price=111,
)

book.replace_order(1, ask_order, new_ask_order) # first arg is agent_id
print("L3 bid data:", book.get_l3_bid_data())
print("L3 ask data:", book.get_l3_ask_data())
print("old_order.order_id:", agent.messages[-1][1].old_order.order_id)
print("new_order.order_id:", agent.messages[-1][1].new_order.order_id)

L3 bid data: [(100, [20, 30])]
L3 ask data: [(111, [50])]
old_order.order_id: 7
new_order.order_id: 12


In [274]:
# book.pretty_print(silent=False)

### Working with real data

In [5]:
# define load paths
raw_itch_load_path = parent_folder_path + '/dataset/raw/ITCH/'
processed_dataset = '03272019.NASDAQ_ITCH50_AAPL_message_proc.npy'
proc_data_dir = os.path.join('dataset/proc/ITCH/full_view/', processed_dataset)
proc_data_dir = parent_folder_path + '/' + proc_data_dir
symbols_load_path = parent_folder_path + '/dataset/symbols/'
symbols_file = sorted(glob(symbols_load_path + '*sp500*.txt'))[0]

# locate raw ITCH data
itch_message_files = sorted(glob(raw_itch_load_path + '*message*.csv'))
itch_book_files = sorted(glob(raw_itch_load_path + '*book*.csv'))
print('found', len(itch_message_files), 'ITCH message files')
print('found', len(itch_book_files), 'ITCH book files')

# create reverse ticker symbol mapping (key is index, value is ticker)
# print(symbols_file)
tickers = {}
with open(symbols_file) as f:
    idx = 0
    for line in f:
        idx += 1
        # tickers[line.strip()] = idx
        tickers[idx] = line.strip()
# print("Number of unique symbols:", len(tickers))

# load raw ITCH data (book)
symbols = []
for m_f, b_f in tqdm(zip(itch_message_files, itch_book_files)):
    if '03272019' not in m_f:
        continue
    # print(m_f)
    print(b_f)
    
    itch_messages = itch_preproc.load_message_df(m_f)

    itch_book = pd.read_csv(
        b_f,
        # index_col=False,
        # header=None
    )
    assert len(itch_messages) == len(itch_book)

    # remove disallowed order types
    allowed_events=['A','E','C','D','R']
    itch_messages = itch_messages.loc[itch_messages.type.isin(allowed_events)]
    # make sure book is same length as messages
    itch_book = itch_book.loc[itch_messages.index]

    # print("ITCH messages shape:", itch_messages.shape)
    print("ITCH book shape:", itch_book.shape)

    # # remove time field from ITCH book data
    # itch_book = itch_book.drop(columns=['time'])

    # symbol to store in list and use to create OB objects in loop later
    symbol = m_f.rsplit('/', maxsplit=1)[-1][:-12].rsplit('_', maxsplit=1)[-1]
    print("Adding symbol:", symbol)
    symbols.append(symbol)

# load processed ITCH data (messages)
proc_messages = np.array(np.load(proc_data_dir, mmap_mode='r'))
assert len(itch_book) == len(proc_messages) + 1 # off by 1 bc of first message
print("proc_messages.shape:", proc_messages.shape)
print("proc_messages:", proc_messages)
print([ "ticker", "order_id",
        "event_type", "direction", "price_abs", "price", "fill_size", "remain_size",
        "delta_t_s", "delta_t_ns", "time_s", "time_ns", "old_id",
        "price_ref", "fill_size_ref", "time_s_ref", "time_ns_ref", "old_price_abs"])

found 8 ITCH message files
found 8 ITCH book files


0it [00:00, ?it/s]

/home/aaron/Documents/Github/MarketSimT/dataset/raw/ITCH/03272019.NASDAQ_ITCH50_AAPL_book_20.csv


8it [00:04,  1.69it/s]

ITCH book shape: (2003141, 81)
Adding symbol: AAPL
proc_messages.shape: (2003140, 18)
proc_messages: [[       40     15969         1 ...     -9999     -9999     -9999]
 [       40     20677         1 ...     -9999     -9999     -9999]
 [       40     22061         1 ...     -9999     -9999     -9999]
 ...
 [       40 335312241         4 ...     56890 566837429     -9999]
 [       40 343725045         4 ...     57309 388685794     -9999]
 [       40 353209329         4 ...     57799 533545153     -9999]]
['ticker', 'order_id', 'event_type', 'direction', 'price_abs', 'price', 'fill_size', 'remain_size', 'delta_t_s', 'delta_t_ns', 'time_s', 'time_ns', 'old_id', 'price_ref', 'fill_size_ref', 'time_s_ref', 'time_ns_ref', 'old_price_abs']


In [6]:
itch_book

,time,1_bid_price,1_bid_vol,1_ask_price,1_ask_vol,2_bid_price,2_bid_vol,2_ask_price,2_ask_vol,3_bid_price,...,18_ask_price,18_ask_vol,19_bid_price,19_bid_vol,19_ask_price,19_ask_vol,20_bid_price,20_bid_vol,20_ask_price,20_ask_vol
0,14400006432545,NaN,NaN,207.85,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14400008777412,129.33,100.0,207.85,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14400016498868,129.33,100.0,207.85,18.0,114.94,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14400017857990,129.33,100.0,207.85,18.0,114.94,1.0,NaN,NaN,98.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14403597489791,129.33,100.0,192.70,300.0,114.94,1.0,207.85,18.0,98.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010131,72000073334596,188.00,1.0,188.99,1000.0,175.00,50.0,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010132,72000073447563,175.00,50.0,188.99,1000.0,NaN,NaN,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010133,72000073477970,NaN,NaN,188.99,1000.0,NaN,NaN,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010134,72000074023439,NaN,NaN,188.99,1000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
proc_messages[0:10]

array([[       40,     15969,         1,         0,     12933,       999,
              100,     -9999,         0,   2344867,     14400,   8777412,
            -9999,     -9999,     -9999,     -9999,     -9999,     -9999],
       [       40,     20677,         1,         0,     11494,      -999,
                1,     -9999,         0,   7721456,     14400,  16498868,
            -9999,     -9999,     -9999,     -9999,     -9999,     -9999],
       [       40,     22061,         1,         0,      9839,      -999,
                1,     -9999,         0,   1359122,     14400,  17857990,
            -9999,     -9999,     -9999,     -9999,     -9999,     -9999],
       [       40,     98453,         1,         1,     19270,       999,
              300,     -9999,         3, 579631801,     14403, 597489791,
            -9999,     -9999,     -9999,     -9999,     -9999,     -9999],
       [       40,     99329,         1,         1,     21490,       999,
               85,     -9999,     

In [8]:
print("Number of unique symbols:", len(tickers))
print(tickers[proc_messages[0][0]])
symbols

Number of unique symbols: 503
AAPL


['AAPL']

In [149]:
def merge_lists(list1, list2):
    '''Merge two lists into one by alternating elements from each list.'''
    list = []
    for item1, item2 in zip(list1, list2):
        list.append(item1)
        list.append(item2)
    return list

def validate_book(sim_book, actual_book, book_idx, n_levels=20):
    # pad and format simulated data
    sim_bids = sim_book.get_l2_bid_data()
    sim_bids += [(0,0)] * (n_levels - len(sim_bids))
    sim_asks = sim_book.get_l2_ask_data()
    sim_asks += [(0,0)] * (n_levels - len(sim_asks))
    sim_book = merge_lists(sim_bids, sim_asks)[:n_levels*2]

    # format actual data
    b_prices = actual_book.iloc[book_idx, 1::2].mul(100).fillna(0).astype(int).values # bid and ask prices
    b_vol = actual_book.iloc[book_idx, 2::2].fillna(0).astype(int).values # bid and ask volumes
    actual_book = list(zip(b_prices, b_vol))

    # compare
    # print(sim_book)
    # print(actual_book)
    assert sim_book == actual_book, f"Book {book_idx} not validated. Simulated book: {sim_book}, Actual book: {actual_book}"
    # print(f"Book {book_idx} validated.")


In [263]:
# init new book under nasdaq agent
nasdaq_agent = FakeExchangeAgent()

# create a dictionary of order books based on each symbol in symbols
order_books = {}
for symbol in symbols:
    order_books[symbol] = OrderBook(nasdaq_agent, symbol)

# empty book
assert order_books[tickers[proc_messages[0][0]]].bids == order_books[tickers[proc_messages[0][0]]].asks == []


In [264]:
# first message is missing in proc_messages, so we'll use the raw message file to start the book
first_message = itch_messages.iloc[0]
print(first_message)

# insert bid order
bid_order = LimitOrder(
    order_id=first_message['id'],
    agent_id=1, # world agent, leave alone for now
    time_placed=first_message['time'],
    symbol=symbols[0],
    quantity=int(first_message['size']),
    side=Side.BID if first_message['side'] == 0 else Side.ASK,
    limit_price=int(first_message['price']*100),
)
order_books[symbols[0]].handle_limit_order(bid_order)

print("L3 bid data:", order_books[symbols[0]].get_l3_bid_data())
print("L3 ask data:", order_books[symbols[0]].get_l3_ask_data())
nasdaq_agent.messages

time        14400006432545
type                     A
id                   13301
side                     1
size                  18.0
price               207.85
cancSize               NaN
execSize               NaN
oldId                  NaN
oldSize                NaN
oldPrice               NaN
mpid                   NaN
Name: 0, dtype: object
L3 bid data: []
L3 ask data: [(20785, [18])]


[(1,
  OrderAcceptedMsg(message_id=369366, order=(Agent 1 @ 1970-01-01 04:00:00) : ASK 18 AAPL @ $207.85))]

In [265]:
# [ "ticker", "order_id",
# "event_type", "direction", "price_abs", "price", "fill_size", "remain_size",
# "delta_t_s", "delta_t_ns", "time_s", "time_ns", "old_id",
# "price_ref", "fill_size_ref", "time_s_ref", "time_ns_ref", "old_price_abs"]

# init variables to keep track of previous time, price, etc.
prev_time = first_message['time']
prev_price = int(first_message['price']*100)
book_idx = 1 # start at 1 because we already handled the first message
# order_dict = {} # store order objects for each order_id; use for ref order types (3, 4, 5)

# iterate through messages and update order books
for msg in proc_messages:
    # print(msg)
    symbol = tickers[msg[0]]
    # print("Symbol:", symbol)
    order_id = msg[1]
    # print("Order ID:", order_id)
    event_type = msg[2]
    # print("Event Type:", event_type)
    # direction = Side.BID if msg[3] == 0 else Side.ASK
    # print("Direction:", direction)
    if abs(msg[5]) < 999: # if price was not truncated or NaN
        # price = msg[5] + prev_price
        # calculate reference price (previous mid price)
        mid_price = (((itch_book.iloc[book_idx-1, 1] * 100) + (itch_book.iloc[book_idx-1, 3] * 100)) / 2) // 1
        price = int(mid_price) + msg[5]
        # TODO: Change mid_price to be calculated from order book object
    else: # use price_abs
        price = msg[4]
    # print("Price:", price)
    # verify time correctness
    assert prev_time + (msg[8]*1000000000) + msg[9] == (msg[10] * 1000000000) + msg[11]
    time = prev_time + (msg[8]*1000000000) + msg[9]
    # print("Time:", time)

    # handle order based on event type
    if event_type == 1:
        # ADD LIMIT ORDER
        direction = Side.BID if msg[3] == 0 else Side.ASK
        # print("Direction:", direction)
        fill_size = msg[6]
        # print("Fill Size:", fill_size)
        order = LimitOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=time,
            symbol=symbol,
            quantity=fill_size,
            side=direction,
            limit_price=price,
        )
        order_books[symbol].handle_limit_order(order)
    elif event_type == 2:
        # EXECUTE ORDER
        fill_size = msg[6]
        # print("Fill Size:", fill_size)
        direction = Side.BID if msg[3] == 1 else Side.ASK # opposite of direction in non-execution messages
        # print("Direction:", direction)
        order = MarketOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=time,
            symbol=symbol,
            quantity=fill_size,
            side=direction, # Buy Order if Side.BID (remove liquidity from ask side), Sell Order if Side.ASK (remove liquidity from bid side)
        )
        order_books[symbol].handle_market_order(order)
    elif event_type == 3:
        # EXECUTE ORDER WITH PRICE DIFFERENT THAN DISPLAY
        # This order type is most likely an execution of a price-to-comply order, which is handled by the simulator
        # but this not encoded in the ITCH data beforehand, so we cannot know whether an order is price-to-comply at the time of submission
        # therefore, we handle this event type as a modifed order and then regular execution order (for now, until we revise the data processing)

        # modfify the matched limit order
        direction = Side.BID if msg[3] == 0 else Side.ASK
        ref_order_time = (msg[15] * 1000000000) + msg[16]
        ref_order_size = msg[14]
        ref_order_price = msg[17] # old_price_abs (not mid_price so we cannot calculate using price_ref msg[13])
        # define original order
        original_order = LimitOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=ref_order_time,
            symbol=symbol,
            quantity=ref_order_size,
            side=direction,
            limit_price=ref_order_price,
        )
        # define modified order
        # modified_price = msg[4]
        modified_order = LimitOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=ref_order_time,
            symbol=symbol,
            quantity=ref_order_size,
            side=direction,
            limit_price=price,
        )
        order_books[symbol].modify_order(original_order, modified_order)
        # execute the modified order
        fill_size = msg[6]
        # print("Fill Size:", fill_size)
        direction = Side.BID if msg[3] == 1 else Side.ASK # opposite of direction in non-execution messages
        # print("Direction:", direction)
        order = MarketOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=time,
            symbol=symbol,
            quantity=fill_size,
            side=direction, # Buy Order if Side.BID (remove liquidity from ask side), Sell Order if Side.ASK (remove liquidity from bid side)
        )
        order_books[symbol].handle_market_order(order)
    elif event_type == 4:
        # CANCEL ORDER
        direction = Side.BID if msg[3] == 0 else Side.ASK
        # print("Direction:", direction)
        ref_order_time = (msg[15] * 1000000000) + msg[16]
        if msg[7] == 0:
            # FULL DELETION
            fill_size = msg[6]
            # print("Cancel Size:", fill_size)
            order = LimitOrder(
                order_id=order_id,
                agent_id=1, # world agent, leave alone for now
                time_placed=ref_order_time,
                symbol=symbol,
                quantity=fill_size,
                # quantity=msg[14], # total size of order when placed
                side=direction,
                limit_price=price,
            )
            order_books[symbol].cancel_order(order)
        else:
            # PARTIAL CANCELLATION
            cancel_size = msg[6]
            # print("Partial Cancel Size:", cancel_size)
            # ref_order_size = msg[14]
            ref_order_size = msg[7] + cancel_size # total size of order before partial cancel
            order = LimitOrder(
                order_id=order_id,
                agent_id=1, # world agent, leave alone for now
                time_placed=ref_order_time,
                symbol=symbol,
                quantity=ref_order_size,
                side=direction,
                limit_price=price,
            )
            order_books[symbol].partial_cancel_order(order, cancel_size)
    elif event_type == 5:
        # REPLACE ORDER
        direction = Side.BID if msg[3] == 0 else Side.ASK
        # print("Direction:", direction)
        old_order_id = msg[12]
        old_order_time = (msg[15] * 1000000000) + msg[16]
        old_order_size = msg[14]
        old_order_price = msg[17] # old_price_abs (not mid_price so we cannot calculate using price_ref msg[13])
        # define old order
        old_order = LimitOrder(
            order_id=old_order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=old_order_time,
            symbol=symbol,
            quantity=old_order_size,
            side=direction,
            limit_price=old_order_price,
        )
        new_order_size = msg[6]
        # define new order
        new_order = LimitOrder(
            order_id=order_id,
            agent_id=1, # world agent, leave alone for now
            time_placed=time,
            symbol=symbol,
            quantity=new_order_size,
            side=direction,
            limit_price=price,
        )
        order_books[symbol].replace_order(1, old_order, new_order) # first arg is agent_id (world agent)
    else:
        raise NotImplementedError("Event type not implemented")

    if order_id == 7819469:
        print("msg:", msg)
        print("Event Type:", event_type)
        sim_bids = order_books[symbol].get_l2_bid_data()
        sim_bids += [(0,0)] * (20 - len(sim_bids))
        sim_asks = order_books[symbol].get_l2_ask_data()
        sim_asks += [(0,0)] * (20 - len(sim_asks))
        sim_book = merge_lists(sim_bids, sim_asks)[:20*2]
        print(f"Simulated book at time {time}: {sim_book}")
        print("Order object quantity:", order.quantity)
        print("Messages order: ", nasdaq_agent.messages[-1][1].order)
        print("Messages order quantity: ", nasdaq_agent.messages[-1][1].order.quantity)
        print("")

    # compare simulated order book with actual order book
    assert int(itch_book.iloc[book_idx].time) == time, f"Time mismatch: {int(itch_book.iloc[book_idx].time)} != {time}"
    validate_book(order_books[symbol], itch_book, book_idx)
    book_idx += 1

    # update previous time and price
    prev_time = time
    prev_price = price

    # # store order object (use for reference order types: 3, 4, 5)
    # order_dict[order_id] = order
    

order.quantity:  400
book[0].visible_orders:  [((Agent 1 @ 1970-01-01 09:30:00) : BID 20 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 27 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 1 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 25 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 4 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 7 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 13 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 100 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 119 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 6 AAPL @ $188.70, {}), ((Agent 1 @ 1970-01-01 09:30:00) : BID 150 AAPL @ $188.70, {})]
visible 0:  7075501
visible 1:  7089453
2
msg: [       40   7819469         1         1     18888        17       400
     -9999         0         0     34200 606488029     -9999     -9999
     -9999     -9999     -9999     -9999]
Event Type: 1
Simulated book a

AssertionError: Book 15545 not validated. Simulated book: [(18883, 100), (18888, 200), (18878, 100), (18889, 100), (18877, 100), (18890, 1673), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18841, 700), (18918, 300), (18833, 50), (18920, 130), (18831, 300), (18923, 30), (18829, 60), (18925, 200), (18827, 1), (18929, 30), (18825, 536), (18930, 10), (18824, 440), (18934, 100)], Actual book: [(18883, 100), (18888, 250), (18878, 100), (18889, 100), (18877, 100), (18890, 1673), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18

**Validation note**
The order book has a mismatch at timestep 34223237674883 (book_idx=15545 not validated) due to an order being executed out of order and subsequently making the volume of a cancel order mismatched with the actual book data.
```
Simulated Book: [(18883, 100), (18888, 200), (18878, 100), (18889, 100), (18877, 100), (18890, 1673), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18841, 700), (18918, 300), (18833, 50), (18920, 130), (18831, 300), (18923, 30), (18829, 60), (18925, 200), (18827, 1), (18929, 30), (18825, 536), (18930, 10), (18824, 440), (18934, 100)]

Actual Book: [(18883, 100), (18888, 250), (18878, 100), (18889, 100), (18877, 100), (18890, 1673), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18841, 700), (18918, 300), (18833, 50), (18920, 130), (18831, 300), (18923, 30), (18829, 60), (18925, 200), (18827, 1), (18929, 30), (18825, 536), (18930, 10), (18824, 440), (18934, 100)]
```
In the messages, ADD ORDER @ 188.88 (order_id=9028925) is submitted at timestep 34099118707287 and ADD ORDER @ 188.88 (order_id=7819469) is submitted later on at timestep 34200606488029. Yet, the later order (order_id=7819469) is partially executed at timestep 34223232709799 (and remaining order quantity deleted at 34223237674883), even though this order was submitted later and lower in priority than order 9028925. This order (9028925) is reported as being executed at timestep 34223330769625, shortly after the deletion of order 7819469. 

This discrepancy may be due to a late reporting error (or perhaps intentionally?) in the message sequence on behalf of NASDAQ, this issue has been identified previously and may be common in NASDAQ ITCH data: https://www.elitetrader.com/et/threads/reconstruction-an-order-book-from-an-itch-feed-am-i-missing-something.247328/. Sometimes executions may be reported late and assigned a special sale condition code so that users know the execution is out of sequence. In the future, we may consider throwing out these kind of orders in the preprocessing step, because there is no way of knowing where in the sequence they truly occur.

One solution for handling this instance (for data validation purposes) would be to handle all order matching, etc. based on the order_id given in the messages rather than the order book's programmed matching algorithm. This is not ideal for several reasons, one reason being that our current implementation of our AI model does not generate the order_ids itself (this is instead handled externally by the simulator). Therefore, for simulation purposes, we will rely on the programmed matching algorithm (rather than order_id based matching) and consider the simulator/order book implemention "sufficiently validated".

**Debugging log:**
```
order.quantity:  400
visible 0:  7075501
visible 1:  7089453
msg: [       40   7819469         1         1     18888        17       400
     -9999         0         0     34200 606488029     -9999     -9999
     -9999     -9999     -9999     -9999]
Event Type: 1
Simulated book at time 34200606488029: [(18870, 472), (18872, 1000), (18858, 2), (18875, 4296), (18849, 31), (18877, 19), (18837, 100), (18880, 310), (18823, 100), (18884, 419), (18821, 400), (18885, 17), (18812, 200), (18888, 450), (18811, 50), (18889, 100), (18810, 500), (18890, 10), (18805, 200), (18894, 300), (18802, 400), (18895, 700), (18800, 460), (18914, 500), (18793, 200), (18916, 300), (18787, 100), (18918, 200), (18785, 100), (18937, 200), (18782, 200), (18938, 300), (18780, 1000), (18945, 100), (18777, 100), (18948, 100), (18760, 100), (18949, 100), (18757, 50), (18956, 100)]
Order object quantity: 400
Messages order:  (Agent 1 @ 1970-01-01 09:30:00) : ASK 400 AAPL @ $188.88
Messages order quantity:  400


order.quantity:  365
book[0].visible_orders:  [((Agent 1 @ 1970-01-01 09:28:19) : ASK 50 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:00) : ASK 400 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:01) : ASK 100 AAPL @ $188.88, {})]
visible 0:  9028925
visible 1:  7819469
order.quantity:  315
book[0].visible_orders:  [((Agent 1 @ 1970-01-01 09:30:00) : ASK 400 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:01) : ASK 100 AAPL @ $188.88, {})]
visible 0:  7819469
visible 1:  9982133
order.quantity:  315
matched_order:  (Agent 1 @ 1970-01-01 09:30:00) : ASK 315 AAPL @ $188.88
matched_order order_id:  7819469
matched_order quantity:  315
matched_order time_placed:  34200606488029
book[0].visible_orders:  [((Agent 1 @ 1970-01-01 09:30:00) : ASK 400 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:01) : ASK 100 AAPL @ $188.88, {})]
msg: [       40   7819469         2         1     18888         4       365
        35         0         0     34223 232709799     -9999        17
       400     34200 606488029     -9999]
Event Type: 2
Simulated book at time 34223232709799: [(18881, 200), (18888, 185), (18878, 100), (18889, 100), (18877, 100), (18890, 1573), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18841, 700), (18918, 300), (18833, 50), (18920, 130), (18831, 300), (18923, 30), (18829, 60), (18925, 200), (18827, 1), (18929, 30), (18825, 536), (18930, 10), (18824, 440), (18934, 100)]
Order object quantity: 365
Messages order:  (Agent 1 @ 1970-01-01 09:30:23) : MKT Order BID 315 AAPL
Messages order quantity:  315


visible:  [((Agent 1 @ 1970-01-01 09:30:00) : ASK 85 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:01) : ASK 100 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:23) : ASK 100 AAPL @ $188.88, {})]
visible 0:  7819469
visible 1:  9982133
visible 2:  10766189
((Agent 1 @ 1970-01-01 09:30:00) : ASK 85 AAPL @ $188.88, {})
(Agent 1 @ 1970-01-01 09:30:00) : ASK 85 AAPL @ $188.88
cancel order order_id:  7819469
cancel order agent_id:  1
cancel order quantity:  85
cancel order side:  Side.ASK
cancel order time_placed:  34200606488029
visible:  [((Agent 1 @ 1970-01-01 09:30:01) : ASK 100 AAPL @ $188.88, {}), ((Agent 1 @ 1970-01-01 09:30:23) : ASK 100 AAPL @ $188.88, {})]
visible 0:  9982133
visible 1:  10766189
hidden:  []
{}
msg: [       40   7819469         4         1     18888         3        35
         0         0   2939403     34223 237674883     -9999        17
       400     34200 606488029     -9999]
Event Type: 4
Simulated book at time 34223237674883: [(18883, 100), (18888, 200), (18878, 100), (18889, 100), (18877, 100), (18890, 1673), (18876, 100), (18893, 1000), (18875, 400), (18895, 700), (18873, 100), (18899, 250), (18871, 100), (18900, 6726), (18870, 100), (18903, 25), (18869, 100), (18906, 100), (18866, 100), (18907, 500), (18860, 200), (18910, 2060), (18845, 500), (18914, 500), (18842, 740), (18916, 400), (18841, 700), (18918, 300), (18833, 50), (18920, 130), (18831, 300), (18923, 30), (18829, 60), (18925, 200), (18827, 1), (18929, 30), (18825, 536), (18930, 10), (18824, 440), (18934, 100)]
Order object quantity: 35
Messages order:  (Agent 1 @ 1970-01-01 09:30:00) : ASK 85 AAPL @ $188.88
Messages order quantity:  85
```

In [168]:
# b_prices = itch_book.iloc[book_idx, 1::2].mul(100).fillna(0).astype(int).values # bid and ask prices
# b_vol = itch_book.iloc[book_idx, 2::2].fillna(0).astype(int).values # bid and ask volumes
# actual_book = list(zip(b_prices, b_vol))

# # b_prices = itch_book.iloc[book_idx-1, 1::2].mul(100).fillna(0).astype(int).values # bid and ask prices
# # b_vol = itch_book.iloc[book_idx-1, 2::2].fillna(0).astype(int).values # bid and ask volumes
# # actual_book = list(zip(b_prices, b_vol))

# actual_book

In [91]:
# order_id

In [208]:
order.order_id
order.quantity

35

In [92]:
# order_books[symbol].history
# direction

In [66]:
itch_book

,time,1_bid_price,1_bid_vol,1_ask_price,1_ask_vol,2_bid_price,2_bid_vol,2_ask_price,2_ask_vol,3_bid_price,...,18_ask_price,18_ask_vol,19_bid_price,19_bid_vol,19_ask_price,19_ask_vol,20_bid_price,20_bid_vol,20_ask_price,20_ask_vol
0,14400006432545,NaN,NaN,207.85,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14400008777412,129.33,100.0,207.85,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14400016498868,129.33,100.0,207.85,18.0,114.94,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14400017857990,129.33,100.0,207.85,18.0,114.94,1.0,NaN,NaN,98.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14403597489791,129.33,100.0,192.70,300.0,114.94,1.0,207.85,18.0,98.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010131,72000073334596,188.00,1.0,188.99,1000.0,175.00,50.0,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010132,72000073447563,175.00,50.0,188.99,1000.0,NaN,NaN,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010133,72000073477970,NaN,NaN,188.99,1000.0,NaN,NaN,195.00,2600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010134,72000074023439,NaN,NaN,188.99,1000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
print("L3 bid data:", order_books[symbols[0]].get_l3_bid_data())
print("L3 ask data:", order_books[symbols[0]].get_l3_ask_data())
nasdaq_agent.messages

L3 bid data: [(18883, [100]), (18878, [100]), (18877, [100]), (18876, [100]), (18875, [100, 300]), (18873, [100]), (18871, [100]), (18870, [100]), (18869, [100]), (18866, [100]), (18860, [200]), (18845, [500]), (18842, [740]), (18841, [700]), (18833, [50]), (18831, [200, 100]), (18829, [60]), (18827, [1]), (18825, [10, 5, 4, 2, 5, 10, 500]), (18824, [400, 30, 10]), (18823, [100, 8]), (18821, [400]), (18820, [15, 2, 23, 75, 3, 1, 1]), (18819, [100]), (18818, [1, 5]), (18817, [15, 41, 1, 10]), (18815, [1, 25]), (18813, [2]), (18811, [50, 1]), (18810, [500, 3, 1, 1, 6, 200]), (18808, [100, 3]), (18806, [25]), (18805, [200, 10]), (18804, [100]), (18802, [300, 10, 8]), (18800, [4, 1, 1, 1, 1, 1, 1, 50, 100, 100, 200, 100, 20, 100, 10, 10, 1, 3, 12, 1, 1, 1, 24, 80, 50, 100, 9, 1, 5, 1, 10, 1, 100, 5, 55, 4, 2, 1, 10, 25, 10, 27, 5, 5, 98, 142, 1000, 95, 100, 60, 60, 100, 10, 48, 6, 1000, 40, 16, 10, 4, 5, 1, 1, 1, 8, 100, 10, 3, 1, 65, 60, 6, 50, 1, 57, 1, 200, 100, 30, 6, 10, 9999, 1, 5, 1

[(1,
  OrderAcceptedMsg(message_id=94986, order=(Agent 1 @ 1970-01-01 04:00:00) : ASK 18 AAPL @ $207.85)),
 (1,
  OrderAcceptedMsg(message_id=94987, order=(Agent 1 @ 1970-01-01 04:00:00) : BID 100 AAPL @ $129.33)),
 (1,
  OrderAcceptedMsg(message_id=94988, order=(Agent 1 @ 1970-01-01 04:00:00) : BID 1 AAPL @ $114.94)),
 (1,
  OrderAcceptedMsg(message_id=94989, order=(Agent 1 @ 1970-01-01 04:00:00) : BID 1 AAPL @ $98.39)),
 (1,
  OrderAcceptedMsg(message_id=94990, order=(Agent 1 @ 1970-01-01 04:00:03) : ASK 300 AAPL @ $192.70)),
 (1,
  OrderAcceptedMsg(message_id=94991, order=(Agent 1 @ 1970-01-01 04:00:03) : ASK 85 AAPL @ $214.90)),
 (1,
  OrderAcceptedMsg(message_id=94992, order=(Agent 1 @ 1970-01-01 04:00:03) : ASK 150 AAPL @ $225.00)),
 (1,
  OrderAcceptedMsg(message_id=94993, order=(Agent 1 @ 1970-01-01 04:00:03) : ASK 5 AAPL @ $225.00)),
 (1,
  OrderAcceptedMsg(message_id=94994, order=(Agent 1 @ 1970-01-01 04:00:05) : BID 25 AAPL @ $185.73)),
 (1,
  OrderAcceptedMsg(message_id=949

In [232]:
nasdaq_agent.messages[-1][1].order.quantity

85

In [210]:
nasdaq_agent.messages[-2][1].order.quantity

100